## Data aquiring date: 10/16/2020

author of code: 
- Frank Zhao
- Gustavo Ferreira
- Jason Li

# Import modules

In [1]:
import json
from pprint import pprint
import glob
from datetime import datetime
import requests
import pandas as pd

# Helper Functions

In [2]:
## function for realtor api requests
### credit https://rapidapi.com/apidojo/api/realtor?endpoint=apiendpoint_e259775d-d98e-479f-8440-206d6d4fa892

def realtor_api_request(sort = "relevance", city = "Philadelphia", limit = "200", offset = "0", state_code = "PA"):
    url = "https://realtor.p.rapidapi.com/properties/v2/list-for-sale"

    querystring = {
        "sort": sort,
        "city": city,
        "limit": limit,
        "offset": offset,
        "state_code": state_code
    }

    headers = {
        'x-rapidapi-host': "realtor.p.rapidapi.com",
        'x-rapidapi-key': "e2aef74c96msh10b708b155c5b67p1e6d19jsn4cb3a156a37d"
        }

    response = requests.request("GET", url, headers=headers, params=querystring)
    
    return response.json()

In [3]:
## convert response to dataFrame

def process_response(response_json):
    '''
    This function is to convert each request result to a dataframe.
    
    1. create an empty list
    2. loop for each response and get details from key 'properties' 
    3. convert details to df
    4. append single df to list
    5. concat the list to one df
    
    '''
    
    convert_list=[]

    for col in response_json['properties']:
        
        single_df = pd.DataFrame.from_dict(col, orient='index').T
        
        convert_list.append(single_df)
        
    return pd.concat(convert_list, axis = 0, ignore_index=True, sort=False)

# Get data

In [4]:
## run for each offset
'''
There are 200 instances in each request. Restricted by Rapidapi
'''
def response_to_df(sort = "relevance", city = "Philadelphia", limit = "200", offset = "0", state_code = "PA"):
    
    property_for_sale = realtor_api_request(sort = sort, 
                                            city = city, 
                                            limit = limit, 
                                            offset = offset, 
                                            state_code = state_code)
    df_properties_for_sale_raw = process_response(response_json = property_for_sale)
    
    pd.set_option('display.max_columns', None)
    
    return df_properties_for_sale_raw


# Export response to csv file

In [5]:
## export each response to csv file
'''
Since each request could get 200 instances, after the each request, we increase the offset 200.
So on the second run, we can get data from 201 - 400, theoratically.
However, the system will keep updating the data so we may either get the data overlapped or miss some data.
We will deal with duplicated data later.

Currently, the loop will crash when the instances in the last run is less than 200. 
'''

start = datetime.now() ## to see how long will it finish the run
i=1
for offset in range(0, 9562, 200):
    response_to_df(offset = offset).to_csv(f'test{i}.csv',encoding='utf-8', index=False)
    if i % 10 == 0:
        print(f'The {i}th Time')
    i += 1    
end = datetime.now() -start

print(f'Total time is: {end}')

The 10th Time
The 20th Time
The 30th Time
The 40th Time
Total time is: 0:02:54.604970
